In [3]:
import torch
import torchvision
import torch.nn as nn  # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim  # For all Optimization algorithms, SGD, Adam, etc.
import torch.nn.functional as F  # All functions that don't have any parameters
import torchvision.datasets as datasets  # Has standard datasets we can import in a nice way
import torchvision.transforms as transforms  # Transformations we can perform on our dataset
from torch.utils.data import (
    DataLoader,
)  # Gives easier dataset managment and creates mini batches
from torch.utils.tensorboard import SummaryWriter  # to print to tensorboarder

In [4]:
# Simple CNN
class CNN(nn.Module):
    def __init__(self, in_channels=1, num_classes=10):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels=in_channels, out_channels=8, kernel_size=3, stride=1, padding=1
        )
        self.pool = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        self.conv2 = nn.Conv2d(
            in_channels=8, out_channels=16, kernel_size=3, stride=1, padding=1
        )
        self.fc1 = nn.Linear(16 * 7 * 7, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)
        return x

In [8]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
in_channels = 1
num_classes = 10
num_epochs = 3
lr = 0.001
batch_size = 64
num_epochs = 5

# Load Data
train_dataset = datasets.MNIST(
    root="dataset/", train=True, transform=transforms.ToTensor(), download=True
)

In [9]:
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

In [11]:
model = CNN(in_channels=in_channels, num_classes=num_classes)
model.to(device)

CNN(
  (conv1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=784, out_features=10, bias=True)
)

In [13]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=0.0)

**Tensorboard**
Write into the current fold's `runs` fold

In [14]:
writer = SummaryWriter(f'runs/MNIST/tryingout_tensorboard')

In [17]:
# adding one batch, adding one more step
step = 0
for epoch in range(num_epochs):
    losses = []
    accuracies = []
    
    for batch_idx, (xs, ys) in enumerate(train_loader):
        xs = xs.to(device=device)
        ys = ys.to(device=device)
        
        # forward
        hat_ys = model(xs)
        loss = loss_function(hat_ys, ys)
        losses.append(loss.item())
        
        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Calculate 'runing' training accuracy
        _, predictions = hat_ys.max(1)
        num_correct = (predictions == ys).sum()
        running_train_acc = float(num_correct)/float(xs.shape[0])
        
        '''
        Adding the end of every batch
        '''
        writer.add_scalar('Training Loss', loss, global_step=step)
        writer.add_scalar('Training Accuracy', running_train_acc, global_step=step)
        step += 1
        
    print(f'Mean loss this epoch was {sum(losses)/len(losses)}')

Mean loss this epoch was 0.3211231522424929
Mean loss this epoch was 0.09592510640386866
Mean loss this epoch was 0.07339006033850147
Mean loss this epoch was 0.06116277782538378
Mean loss this epoch was 0.052735856733843886


**Notes**

Open terms

- `Tensorboard --logdir runs` runs is the stored